# Assignment 4

In [1]:
import numpy as np   # Do not modify these imports
import pandas as pd
import math

### Part A

A bond with a face value of ```FV``` and a coupon rate of ```cpn_rate``` will mature `yrs` years and `mts` months from today. The bond pays coupons semi-annually, with the next coupon to be paid 3 months from today. Suppose the yield to maturity of this bond is ```r``` (APR, compounded semi-annually). The parameters `yrs` and `mts` will be non-negative integers, with `mts < 12`.

Write the function ```clean_price```, which returns the clean price of the bond today.

In [17]:
def clean_price(FV, cpn_rate, r, yrs, mts):
    # Write your code here
    cpn = cpn_rate*FV/2 # semi-annual payments
    frac_cpn = 0
    if mts>=9:
        frac_cpn=2
    elif mts>=3:
        frac_cpn=1
    
    N = yrs*2+frac_cpn
    y = r/2 # adjust for APR
    price = ((cpn/y)*(1-1/math.pow(1+y,N))+FV/(math.pow(1+y,N)))
    price *= math.pow((1+y),(mts%6)/6) # adjust for f elapsed coupon period
    acc_interest = cpn*((mts%6)/6)
    price -= acc_interest
    return price

# print(clean_price(1000,0.06,0.058,3,3)) # 1,005.74


1005.7402358063853


Suppose that a zero-coupon bond with a face value of ```FV``` costs ```C1``` and will mature 2 years from today. An otherwise identical bond with maturity 3 years from today is selling for ```C2```. Write the function ```forward_rate```, which calculates the forward rate available today for a period of one year starting 2 years from today.

In [24]:
def forward_rate(FV, C1, C2):
    # Write your code here
    r2=math.pow(FV/C1, 1/2)-1
    r3=math.pow(FV/C2, 1/3)-1
    rf = math.pow((1+r3),3)/math.pow((1+r2),2)-1
    return rf
forward_rate(1000, 902.21, 852.34) #5.8510

0.05850951498228385

### Part B

We wish to study the relation between bond rating and bond yield. 

We will first convert S&P ratings to numerical numbers as follows. Assign 1 to the lowest possible rating, and assign 2 to the second lowest rating, etc. We need to do the same for Moody’s ratings. Please use the conversion functions (```SP_convert```,```moody_convert```)  that are already provided below.

Note that for all following functions, we will only be considering bonds with valid ratings. In other words, if any of these conversion functions returns -1 (signifying an invalid rating), we will not consider it in our calculations. Such bonds should be removed from the data.

In [30]:
# Converts an S&P rating to a number described by the system above.
# Converts all invalid ratings to -1.
def SP_convert(rating):
    if rating == "D":
        return 0
    if rating == "C":
        return 1
    if rating == "CC":
        return 2
    if rating == "CCC-":
        return 3
    if rating == "CCC":
        return 4
    if rating == "CCC+":
        return 5
    if rating == "B-":
        return 6
    if rating == "B":
        return 7
    if rating == "B+":
        return 8
    if rating == "BB-":
        return 9
    if rating == "BB":
        return 10
    if rating == "BB+":
        return 11
    if rating == "BBB-":
        return 12
    if rating == "BBB":
        return 13
    if rating == "BBB+":
        return 14
    if rating == "A-":
        return 15
    if rating == "A":
        return 16
    if rating == "A+":
        return 17
    if rating == "AA-":
        return 18
    if rating == "AA":
        return 19
    if rating == "AA+":
        return 20
    if rating == "AAA":
        return 21
    else:
        return -1

# Converts Moody's ratings to a number described by the system above.
# Converts all invalid ratings to -1.
def moody_convert(rating):
    if rating == "C":
        return 0
    if rating == "Ca":
        return 1
    if rating == "Caa3":
        return 2
    if rating == "Caa2":
        return 3
    if rating == "Caa1":
        return 4
    if rating == "B3":
        return 5
    if rating == "B2":
        return 6
    if rating == "B1":
        return 7
    if rating == "Ba3":
        return 8
    if rating == "Ba2":
        return 9
    if rating == "Ba1":
        return 10
    if rating == "Baa3":
        return 11
    if rating == "Baa2":
        return 12
    if rating == "Baa1":
        return 13
    if rating == "A3":
        return 14
    if rating == "A2":
        return 15
    if rating == "A1":
        return 16
    if rating == "Aa3":
        return 17
    if rating == "Aa2":
        return 18
    if rating == "Aa1":
        return 19
    if rating == "Aaa":
        return 20
    else:
        return -1
    

# The next two functions are variants of the above that can be run on columns.

def SP_convert_column(column):  
    tmp = np.vectorize(SP_convert)
    return pd.Series(tmp(column))

def moody_convert_column(column):
    tmp = np.vectorize(moody_convert)
    return pd.Series(tmp(column))

Write the function ```SP_investment_grade```, which  which takes a path to the ratings csv file (similar to previous assignments), and returns the percentage of bonds with investment grades according to S&P. 

Please see Table 6.4 in the textbook to determine which ratings correspond to investment grade. 

For this analysis, two files are provided (which are originally supplised by the data vendor, FISD). Please see the attached PDF for more data descriptions. More specifically, please see Pg31-32: “Rating” for "mod4_ratings.csv" file and pg 19-23: “Issue” for "mod4_bonds.csv" file. 

In [48]:
def SP_investment_grade(path):
    # Write your code here
    df = pd.read_csv(path)
    df['Converted Rating'] = SP_convert_column(df['RATING'])

    df = df[df['Converted Rating'] != -1] # drop -1 vals
    df = df.reset_index()

    # According to table 6.4 anything >= BBB- is considered investment grade -> gr>=12
    count_gr = 0
    count_t = 0 
    for i in df['Converted Rating']:
        count_t+=1
        if i>=12:
            count_gr+=1
    return count_gr/count_t

SP_investment_grade('mod4_ratings.csv')



0.9326815103841748

Write the function ```moody_investment_grade```, which does the same thing as ```SP_investment_grade```, but according to Moody's instead.

In [50]:
def moody_investment_grade(path):
    # Write your code here
    df = pd.read_csv(path)
    df['Converted Rating'] = moody_convert_column(df['RATING'])

    df = df[df['Converted Rating'] != -1] # drop -1 vals
    df = df.reset_index()
    
    # According to table 6.4 anything >= Baa is considered investment grade -> gr>=11
    count_gr = 0
    count_t = 0 
    for i in df['Converted Rating']:
        count_t+=1
        if i>=11:
            count_gr+=1
    
    return count_gr/count_t

moody_investment_grade('mod4_ratings.csv')

0.9174608707017677

Using the conversion system described above, write the function ```SP_correlation```, which finds the correlation between between S&P rating and bond yield. This function will take two arguments, ```bonds_path```, which is a file path to the bonds csv, and ```ratings_path```, which is a file path to the ratings csv. 

In the same manner, write the function ```moody_correlation```, which does the same thing as ```SP_correlation```, but using the Moody Rating instead.

"OFFERING_YIELD" column sometimes contains unreasonable numbers. And that is most likely due to data glitch. Please address the data glitch problem by setting "OFFERING_YIELD" to 30 if it is larger than 30. 

In this question, you may find it helpful to merge two DataFrames. You can achieve this with the line

`pd.merge(df_1, df_2, how = 'inner', on = ["Column Name"])`

This merges the two dataframes `df_1` and `df_2` based on matching values in the column with name "Column Name", returning the value of the merged DataFrame.

In [67]:
def SP_correlation(bonds_path, ratings_path):
    bonds_df = pd.read_csv(bonds_path)
    ratings_df = pd.read_csv(ratings_path) # Do not change the first 3 lines of this function.
    
    # Write your code here
    df = pd.merge(bonds_df, ratings_df, how = 'inner', on = ["ISSUE_ID"])
    df['Converted Rating'] = SP_convert_column(df['RATING'])
    df = df[df['Converted Rating'] != -1] # drop -1 vals
    df.loc[df['OFFERING_YIELD'] > 30, 'OFFERING_YIELD'] = 30
    df.dropna(inplace=True)
    return df['Converted Rating'].corr(df['OFFERING_YIELD'])

SP_correlation('mod4_bonds.csv','mod4_ratings.csv')

-0.3031620985445146

In [68]:
def moody(bonds_path, ratings_path):
    bonds_df = pd.read_csv(bonds_path)
    ratings_df = pd.read_csv(ratings_path) # Do not change the first 3 lines of this function.
    
    # Write your code here
    df = pd.merge(bonds_df, ratings_df, how = 'inner', on = ["ISSUE_ID"])
    df['Converted Rating'] = moody_convert_column(df['RATING'])
    df = df[df['Converted Rating'] != -1] # drop -1 vals
    df.loc[df['OFFERING_YIELD'] > 30, 'OFFERING_YIELD'] = 30
    df.dropna(inplace=True)
    display(df)
    return df['Converted Rating'].corr(df['OFFERING_YIELD'])

moody('mod4_bonds.csv','mod4_ratings.csv')


ISSUE_ID  ISSUER_ID  MATURITY  OFFERING_DATE  OFFERING_YIELD  \
1          99164       1771   15741.0        14803.0         6.46557   
11        102786       1781   16451.0        14615.0         6.75000   
12        102787       1781   18277.0        14615.0         7.00000   
13        102788       1781   19007.0        14615.0         7.55000   
19        102848       1489   15364.0        14612.0         6.75000   
...          ...        ...       ...            ...             ...   
439867   1018589      51507   24372.0        22544.0         3.18209   
439886   1026058       1497   23937.0        22089.0         0.77000   
439918   1029976      51593   33433.0        22495.0         2.63300   
439925   1030103      51605   33099.0        22124.0         2.51100   
439964   1042525      40720   24684.0        22118.0         0.94700   

       RATING_TYPE RATING_DATE RATING  Converted Rating  
1               MR   10aug1999    Aaa                20  
11              MR   06jan2000     A2                15  
12              MR   06jan2000     A2                15  
13              MR   06jan2000     A2                15  
19              MR   03jan2000    Aaa                20  
...            ...         ...    ...               ...  
439867          MR   20jan2022   Baa3                11  
439886          MR   23jun2020    Aaa                20  
439918          MR   15jul2021    Aa2                18  
439925          MR   16jul2020    Aa3                17  
439964          MR   08jul2020    Aaa                20  

[64263 rows x 9 columns]

-0.4374276094438931